In [8]:
# 필요 시 주석을 해제하고 아래 라이브러리들을 설치
# !pip install pystan==2.19.1.1
# !pip3 install fbprophet
# !pip install plotly

In [3]:
# 라이브러리 임포트
import os
import time
import json
import datetime
import numpy as np
import pandas as pd
from fbprophet import Prophet
import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING) 

In [4]:
# 데이터 로드
train = json.load(open('../data/ai_contest/02.Training_Data/train.json', 'r', encoding='utf8')) 
submit = json.load(open('../data/ai_contest/02.Training_Data/sample_submission.json', 'r', encoding='utf8')) 

In [5]:
df_dic = {}
plc_lst = list(train.keys())
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

In [6]:
plc_lst

['서울서남하수',
 '부산서부하수',
 '부산정관하수',
 '부산동부하수',
 '부산강변하수',
 '부산남부하수',
 '부산중앙하수',
 '부산기장하수',
 '부산해운대하수',
 '부산영도하수',
 '대구지산하수',
 '대구달서천하수',
 '대구북부하수',
 '대구서부하수',
 '대구신천하수',
 '대구안심하수',
 '대구달성폐수',
 '대구성서폐수',
 '인천운북하수',
 '인천공촌하수',
 '인천만수하수',
 '인천강화하수',
 '중부발전(인천)',
 '포스코파워(인천)',
 '광주제2하수',
 '광주제1하수',
 '대전하수',
 '대전대덕폐수',
 '울산온산하수',
 '울산방어진하수',
 '울산언양하수',
 '울산용연하수',
 '울산회야하수',
 '삼성SDI(울산)',
 '효성(울산)',
 '부천굴포천하수',
 '부천역곡하수',
 '평택팽성하수',
 '성남하수',
 '수원하수',
 '오산1하수',
 '고양벽제하수',
 '고양일산하수',
 '화성조암하수',
 '화성남양하수',
 '안산대부하수',
 '안산하수1단계',
 '안산하수2단계',
 '시흥시 맑은물관리센터',
 '구리2하수',
 '의정부하수',
 '양평하수',
 '고양원능하수',
 '의왕부곡하수',
 '포천하수',
 '포천소흘하수',
 '연천하수',
 '안성하수',
 '안성불당하수',
 '용인구갈하수',
 '파주금촌하수',
 '파주통일동산하수',
 '이천장호원하수',
 '이천하수',
 '여주하수',
 '광주경안하수',
 '광주오포하수',
 '과천하수',
 '남양주진건하수',
 '광주곤지암하수',
 '동두천하수',
 '남양주1화도하수',
 '오산2하수',
 '평택현덕하수',
 '구리1하수',
 '가평하수',
 '가평청평하수',
 '가평현리하수',
 '남양주가운하수',
 '연천신서하수',
 '양평용문하수',
 '양평양서하수',
 '여주가남하수',
 '평택포승하수',
 '파주적성하수',
 '용인영덕하수',
 '군포대야하수',
 '안성진사하수',
 '판교하수',
 '용인남사하수',
 '양주남면하수',
 '

In [7]:
# 사업장별 dataframe 제작
for i in plc_lst:
    plc_df = pd.DataFrame(columns={'ds', 'pH', 'COD', 'SS', 'N', 'P', 'T'})
    plc_df = plc_df[['ds', 'pH', 'COD', 'SS', 'N', 'P', 'T']]
    
    date_lst = list(train[i].keys())
    plc_df.loc[:, 'ds'] = date_lst

    for index, j in enumerate(date_lst):
        for k in fct_lst:
            try:
                plc_df.loc[index, k] = train[i][j][k]
            except:
                pass
                    
    df_dic[i] = plc_df

In [8]:
plc_lst[:5]

['서울서남하수', '부산서부하수', '부산정관하수', '부산동부하수', '부산강변하수']

In [9]:
df_dic[plc_lst[0]]

,ds,pH,COD,SS,N,P,T
0,20100101,0.483936,1.27378,1.36436,1.26818,1.1489,0.115951
1,20100102,0.483936,1.15002,1.28054,1.39228,0.544579,0.115951
2,20100103,0.877648,0.94375,1.02908,1.36308,0.678874,0.115951
3,20100104,0.877648,0.861242,1.61583,1.90386,1.23549,0.115951
4,20100105,0.877648,0.654972,1.1129,2.23586,1.40336,0.115951
...,...,...,...,...,...,...,...
2815,20180127,2.05879,0.366194,1.44819,2.20082,-1.02455,-0.136248
2816,20180128,2.05879,0.32494,1.61583,1.96343,-0.983906,-0.136248
2817,20180129,2.05879,0.366194,1.53201,1.93364,-1.01041,-0.136248
2818,20180130,1.66507,1.10877,1.53201,2.45048,-0.992741,-0.136248


In [10]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31'))], columns=['ds'])
future['y'] = np.nan
future

,ds,y
0,2018-02-01,NaN
1,2018-02-02,NaN
2,2018-02-03,NaN
3,2018-02-04,NaN
4,2018-02-05,NaN
...,...,...
694,2019-12-27,NaN
695,2019-12-28,NaN
696,2019-12-29,NaN
697,2019-12-30,NaN


In [11]:
fct_df = plc_df[['ds', j]]
fct_df = fct_df.rename(columns={j:'y'})

KeyError: "['20180131'] not in index"

In [17]:
# 추론
start_time = time.time()

for i in plc_lst:
    plc_df = df_dic[i]
    
    for j in fct_lst:
        fct_df = plc_df[['ds', j]]
        fct_df = fct_df.rename(columns={j:'y'})
        model = Prophet()
        model.fit(fct_df)
        forecast = model.predict(future)
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast = forecast[['ds', 'yhat']]
        
        for index, k in enumerate(list(forecast['ds'])):
            submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

: 

: 

In [ ]:
# 추론 파일 저장
with open('../data/ai_contest/prediction_base.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)

: 